In [ ]:
%pip install -r modules/requirements_ph.txt
from modules.ph import *
from scipy.stats import sem
%pip install parse
import parse

In [ ]:
df_data = pd.read_excel('data_kapazitaet.xlsx')
df_data

In [ ]:
# Einheiten parsen
unit_dict = {parse.parse("{groesse} in {unit}", column).named['groesse']: pe(parse.parse("{groesse} in {unit}", column).named['unit']) for column in df_data.columns}
unit_dict

In [ ]:
# Mittelwerte und Fehler berechnen
df_eval = df_data.groupby(['Spannung in V', 'Abstand d in mm', 'Fläche in cm**2'])['Ladung in C'].agg(['mean', 'sem']).reset_index()
df_eval['Ladung in C'] = df_eval.apply(lambda k: ufloat(k['mean'],k['sem']), axis=1)
df_eval

In [ ]:
# Proportionalitäten untersuchen
A=df_eval.loc[(df_eval['Abstand d in mm'] == 4) & (df_eval['Fläche in cm**2'] == 250) & (df_eval['Spannung in V'] == 200), 'Ladung in C'].values[0]
B=df_eval.loc[(df_eval['Abstand d in mm'] == 2) & (df_eval['Fläche in cm**2'] == 250) & (df_eval['Spannung in V'] == 200), 'Ladung in C'].values[0]

A/B

In [ ]:
# Auswertung mit Regression
df_plot = df_eval[(df_eval['Abstand d in mm'] == 4) & (df_eval['Fläche in cm**2'] == 250)]
df_plot

In [ ]:
def model(p, x):
    a,b = p
    return a*x+b
startwerte=[1,1]

In [ ]:
x_key = 'Spannung in V'
y_key = 'mean'

x_err = 5
y_err = df_plot['sem'].values

fig,ax = plt.subplots(figsize=(10,5))
df_plot.plot(ax=ax,x=x_key,y=y_key, xlabel=f'Spannung in V', ylabel=f'Ladung in C',style='.',xerr=x_err, yerr=y_err, linestyle='None')
ax.grid(True)
ax.legend().set_visible(False)

X = df_plot[x_key]
Y = df_plot[y_key]
up , p, yfit, out = uODR(model,startwerte,X,Y,sx=x_err,sy=df_plot['sem'].values)
X_fit = np.linspace(X.values[0],X.values[-1],100)
Y_fit = model(p, X_fit)
ax.plot(X_fit,Y_fit)
#fig.savefig('plot.pdf',dpi=600)

In [ ]:
results = {
            'a' : up[0] * (unit_dict['Ladung']/unit_dict['Spannung']).to('pF'),
            'b' : up[1] * unit_dict['Ladung'].to('nC'),
          }
for idx,el in results.items():
    print(f"{idx}={el}")